In [1]:
import os
import pandas as pd

In [6]:
import os
import pandas as pd

# Specify the directory where your files are located
input_directory_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Test Folder'

# Specify the directory where you want to save the new Excel files
output_directory_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Test Folder'

# Get a list of all Excel files in the specified directory
file_names = [f for f in os.listdir(input_directory_path) if f.endswith('.xlsx')]

# Create an empty list to store DataFrames for each year
dfs_by_year = []

# Iterate through each file
for file_name in file_names:
    # Construct the full input file path
    input_file_path = os.path.join(input_directory_path, file_name)

    # Load the data for the current file
    df = pd.read_excel(input_file_path)

    # Convert relevant columns to numeric
    numeric_columns = ['Number_of_Applications', 'Number_of_Successes', 'Number_of_Points']
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate Real_Entries
    df['Real_Entries'] = df['Number_of_Points'] ** 2 + 1

    # Calculate Real_Total_Entries
    df['Real_Total_Entries'] = df['Real_Entries'] * df['Number_of_Applications']

    # Calculate Real_Draw_Odds
    df['Real_Draw_Odds'] = df['Real_Entries'] / df.groupby(['District','Year'])['Real_Total_Entries'].transform('sum')

    # Calculate the probability of not drawing any applicant in a single draw
    df['Prob_Not_Drawing'] = (df.groupby(['District','Year'])['Real_Total_Entries'].transform('sum') - df['Real_Entries']) / df.groupby(['District','Year'])['Real_Total_Entries'].transform('sum')

    # Calculate the probability of not drawing any applicant in all draws
    df['Prob_Not_Drawing_All'] = df['Prob_Not_Drawing'] ** df.groupby(['District','Year'])['Number_of_Successes'].transform('sum')

    # Calculate the probability of drawing at least one applicant
    df['Prob_Drawing_At_Least_One'] = 1 - df['Prob_Not_Drawing_All']


    # Add the processed DataFrame to the list
    dfs_by_year.append(df)

    # Construct the full output file path
    output_file_path = os.path.join(output_directory_path, f'{file_name.split(".")[0]}_Processed.xlsx')

    # Save the updated DataFrame to the new Excel file
    df.to_excel(output_file_path, index=False)

# Concatenate all DataFrames into a single DataFrame
full_data = pd.concat(dfs_by_year, ignore_index=True)

# Display the processed data
print(full_data.head())


  Item_Description District Residency  Number_of_Points  \
0       DEERPERMIT   204-50  RESIDENT                 0   
1       DEERPERMIT   204-50  RESIDENT                 0   
2       DEERPERMIT   204-50  RESIDENT                 0   
3       DEERPERMIT   204-50  RESIDENT                 0   
4       DEERPERMIT   204-50  RESIDENT                 0   

   Number_of_Applications  Number_of_Successes  %_Successful  Year  \
0                     169                  169         100.0  2016   
1                      38                   38         100.0  2016   
2                      35                   35         100.0  2017   
3                     130                  130         100.0  2017   
4                      24                   24         100.0  2018   

   Real_Entries  Real_Total_Entries  Real_Draw_Odds  Prob_Not_Drawing  \
0             1                 169        0.003460          0.996540   
1             1                  38        0.003460          0.996540   
2    

In [14]:
import os
import pandas as pd

# Specify the directory where your files are located
input_directory_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Test Folder'

# Specify the directory where you want to save the new Excel files
output_directory_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Test Folder'

# Get a list of all Excel files in the specified directory
file_names = [f for f in os.listdir(input_directory_path) if f.endswith('.xlsx')]

# Create an empty list to store DataFrames for each year
dfs_by_year = []

# Iterate through each file
for file_name in file_names:
    # Construct the full input file path
    input_file_path = os.path.join(input_directory_path, file_name)

    # Load the data for the current file
    df = pd.read_excel(input_file_path)

    # Convert relevant columns to numeric
    numeric_columns = ['Number_of_Applications', 'Number_of_Successes', 'Number_of_Points']
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate Real_Entries
    df['Real_Entries'] = df['Number_of_Points'] ** 2 + 1

    # Calculate Real_Total_Entries
    df['Real_Total_Entries'] = df['Real_Entries'] * df['Number_of_Applications']
    
    df['Nonresident_Percentage'] = 0.1
    
    # Adjust Number_of_Successes based on residency and the actual nonresident percentage
    df['Adjusted_Number_of_Successes'] = df.groupby(['Item_Description','District','Year'])['Number_of_Successes'].transform('sum')
    df['Adjusted_Number_of_Successes'] *= df['Nonresident_Percentage'].where(df['Residency'] == 'NONRESIDENT', 1)
    
    df.loc[(df['Residency'] == 'NONRESIDENT') & (df['Adjusted_Number_of_Successes'] == 0), 'Adjusted_Number_of_Successes'] = 1
    
    # Calculate the probability of not drawing any applicant in a single draw
    df['Prob_Not_Drawing'] = (df.groupby(['Item_Description','District','Year'])['Real_Total_Entries'].transform('sum') - df['Real_Entries']) / df.groupby(['Item_Description','District','Year'])['Real_Total_Entries'].transform('sum')

    # Calculate the probability of not drawing any applicant in all draws
    df['Prob_Not_Drawing_All'] = (df['Prob_Not_Drawing'] ** df['Adjusted_Number_of_Successes'])

    # Calculate the probability of drawing at least one applicant
    df['Prob_Drawing_At_Least_One'] = 1 - df['Prob_Not_Drawing_All']

    # Add the processed DataFrame to the list
    dfs_by_year.append(df)

    # Construct the full output file path
    output_file_path = os.path.join(output_directory_path, f'{file_name.split(".")[0]}_Processed2.0.xlsx')

    # Save the updated DataFrame to the new Excel file
    df.to_excel(output_file_path, index=False)

# Concatenate all DataFrames into a single DataFrame
full_data = pd.concat(dfs_by_year, ignore_index=True)

# Display the processed data
print(full_data.head())


  Item_Description District Residency  Number_of_Points  \
0       DEERPERMIT   204-50  RESIDENT                 0   
1       DEERPERMIT   204-50  RESIDENT                 0   
2       DEERPERMIT   204-50  RESIDENT                 0   
3       DEERPERMIT   204-50  RESIDENT                 0   
4       DEERPERMIT   204-50  RESIDENT                 0   

   Number_of_Applications  Number_of_Successes  %_Successful  Year  \
0                     169                  169         100.0  2016   
1                      38                   38         100.0  2016   
2                      35                   35         100.0  2017   
3                     130                  130         100.0  2017   
4                      24                   24         100.0  2018   

   Real_Entries  Real_Total_Entries  Nonresident_Percentage  \
0             1                 169                     0.1   
1             1                  38                     0.1   
2             1                  35

In [31]:
import pandas as pd

file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Test Folder\Updated_Data.xlsx'

# Load the 'Updated_Data' file into a DataFrame
df = pd.read_excel(file_path)

# Calculate Real_Entries
df.loc[df['Real_Entries'].isna(), 'Real_Entries'] = df['Number_of_Points'] ** 2 + 1

# Calculate Real_Total_Entries
df.loc[df['Real_Total_Entries'].isna(), 'Real_Total_Entries'] = df['Real_Entries'] * df['Number_of_Applications']

# Set Number_of_Applications, Number_of_Successes, %_Successful to 0 for newly added rows
df.loc[df['Real_Total_Entries'].isna(), ['Number_of_Applications', 'Number_of_Successes', '%_Successful']] = 0

# Set Real_Total_Entries to 0 for newly added rows
df.loc[df['Real_Total_Entries'].isna(), 'Real_Total_Entries'] = 0

# Display the updated DataFrame
print(df.head())

# Save the updated DataFrame to the same Excel file
df.to_excel(file_path, index=False)


  Item_Description District Residency  Year  Number_of_Points  \
0       DEERPERMIT   204-50  RESIDENT  2016                 0   
1       DEERPERMIT   204-50  RESIDENT  2016                 0   
2       DEERPERMIT   204-50  RESIDENT  2016                 1   
3       DEERPERMIT   204-50  RESIDENT  2016                 1   
4       DEERPERMIT   204-50  RESIDENT  2016                 2   

   Number_of_Applications  Number_of_Successes  %_Successful  Real_Entries  \
0                     169                  169         100.0             1   
1                      38                   38         100.0             1   
2                       7                    7         100.0             2   
3                       7                    7         100.0             2   
4                       0                    0           0.0             5   

   Real_Total_Entries  Real_Draw_Odds  Prob_Not_Drawing  Prob_Not_Drawing_All  \
0                 169         0.00346           0.99654    

In [32]:
import pandas as pd

# Load the 'Updated_Data' file into a DataFrame
file_path = r'C:\Users\Sam Beighle\Documents\Capstone - FWP Draw\Test Folder\Updated_Data.xlsx'
df = pd.read_excel(file_path)

# Fill in the remaining blanks with the provided calculations
df['Nonresident_Percentage'] = 0.1

# Adjust Number_of_Successes based on residency and the actual nonresident percentage
df['Adjusted_Number_of_Successes'] = df.groupby(['Item_Description', 'District', 'Year'])['Number_of_Successes'].transform('sum')
df['Adjusted_Number_of_Successes'] *= df['Nonresident_Percentage'].where(df['Residency'] == 'NONRESIDENT', 1)

df.loc[(df['Residency'] == 'NONRESIDENT') & (df['Adjusted_Number_of_Successes'] == 0), 'Adjusted_Number_of_Successes'] = 1

# Calculate the probability of not drawing any applicant in a single draw
df['Prob_Not_Drawing'] = (df.groupby(['Item_Description', 'District', 'Year'])['Real_Total_Entries'].transform('sum') - df['Real_Entries']) / df.groupby(['Item_Description', 'District', 'Year'])['Real_Total_Entries'].transform('sum')

# Calculate the probability of not drawing any applicant in all draws
df['Prob_Not_Drawing_All'] = (df['Prob_Not_Drawing'] ** df['Adjusted_Number_of_Successes'])

# Calculate the probability of drawing at least one applicant
df['Prob_Drawing_At_Least_One'] = 1 - df['Prob_Not_Drawing_All']

# Display the updated DataFrame
print(df.head())

# Save the updated DataFrame to the same Excel file
df.to_excel(file_path, index=False)


  Item_Description District Residency  Year  Number_of_Points  \
0       DEERPERMIT   204-50  RESIDENT  2016                 0   
1       DEERPERMIT   204-50  RESIDENT  2016                 0   
2       DEERPERMIT   204-50  RESIDENT  2016                 1   
3       DEERPERMIT   204-50  RESIDENT  2016                 1   
4       DEERPERMIT   204-50  RESIDENT  2016                 2   

   Number_of_Applications  Number_of_Successes  %_Successful  Real_Entries  \
0                     169                  169         100.0             1   
1                      38                   38         100.0             1   
2                       7                    7         100.0             2   
3                       7                    7         100.0             2   
4                       0                    0           0.0             5   

   Real_Total_Entries  Real_Draw_Odds  Prob_Not_Drawing  Prob_Not_Drawing_All  \
0                 169         0.00346          0.996540    